# Setup

In [2]:
import os

os.environ["AZURE_OPENAI_ENDPOINT"] = "https://dalle3-swo.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "e51119f8d8774069a6594d92ccf7a70d"

In [3]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_core.output_parsers import BaseOutputParser
from typing import List, Optional
import re
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0
) 

gpt_35_turbo_16k = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
) 

smart_llm = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-4",
    temperature=0
) 
gpt_35_turbo_instruct = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo-instruct",
    temperature=0
) 
# Embedding
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-02-15-preview",
)

# Current Problems

- câu trả lời bị ngắn
- thỉnh thoảng thiếu: due to the splitter is not good

# Test questions

In [18]:
question_lists = [
    "How can students/parents receive the timetables?",
    "Students have been placed on academic probation, what does this mean?",
    "What if BUV students want to exchange or transfer to other Universities around the world?",
    "Nếu sinh viên BUV muốn tham gia chương trình Trao đổi và Chuyển tiếp sinh viên tới các trường Đại học trên thế giới thì sao?",
    "Is the Internship programme at BUV compulsory to students?",
    "Chương trình Thực tập tại BUV có bắt buộc đối với sinh viên không?"
]

# Method 

# Implementation

## Extract and Create document

```python
Document({
    "page_content": Question, 
    "metadata": {"Answer": "Answer"}
})
```

## Retriever

In [20]:
# load from disk
vectorstore = Chroma(persist_directory="./chroma_db/BUV_QnA", embedding_function=embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [19]:
question_lists

['How can students/parents receive the timetables?',
 'Students have been placed on academic probation, what does this mean?',
 'What if BUV students want to exchange or transfer to other Universities around the world?',
 'Nếu sinh viên BUV muốn tham gia chương trình Trao đổi và Chuyển tiếp sinh viên tới các trường Đại học trên thế giới thì sao?',
 'Is the Internship programme at BUV compulsory to students?',
 'Chương trình Thực tập tại BUV có bắt buộc đối với sinh viên không?']

In [31]:
a = retriever.invoke(question_lists[5])

In [36]:
a[0].metadata['answer']

'BUV study programmes encourage and enable students to take up internships in their semester break every year. From October 2021, BUV has implemented the compulsory Personal and Social Growth Programme (PSG) where students are required to have at least 1 internship in their PSG Transcript along with other activities to complete the degree and be awarded with the PSG Completion Certificate upon graduation.\n\nProfessional Experience Evaluation Form provided and endorsed by Employer and approved by BUV Management is required for each student after each internship, so that the internship is recorded properly in the student’s PSG Transcript.\n\nInternship Completion Certificate is not compulsory for the PSG Transcript and only available upon request.\n\nSome modules within the Staffordshire University’s academic programme also require students to complete work placements and internships to complete the module’s assessment, which is compulsory and must follow specific requirements of the mo

## Chain

In [37]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# get answer in metadata
def format_docs(docs):
    return "\n\n".join(doc.metadata['answer'] for doc in docs)


template = """
You are an AI assistant that helps students answering specific questions based on provided context. 

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

Context:
{context}

Question: 
{question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

from operator import itemgetter
rag_chain_with_vectorstore_retriever = (
    {"context": itemgetter("question") | retriever | format_docs, "question": itemgetter("question")}
    | custom_rag_prompt
    | gpt_35_turbo_16k
    | StrOutputParser()
)

# Testing

In [39]:
question_lists

['How can students/parents receive the timetables?',
 'Students have been placed on academic probation, what does this mean?',
 'What if BUV students want to exchange or transfer to other Universities around the world?',
 'Nếu sinh viên BUV muốn tham gia chương trình Trao đổi và Chuyển tiếp sinh viên tới các trường Đại học trên thế giới thì sao?',
 'Is the Internship programme at BUV compulsory to students?',
 'Chương trình Thực tập tại BUV có bắt buộc đối với sinh viên không?']

In [40]:
rag_chain_with_vectorstore_retriever.invoke({'question':question_lists[0]})

'Students/parents can receive the timetables by providing their correct email addresses to the Student Information Office via studentservice@buv.edu.vn. Thanks for asking!'

In [41]:
rag_chain_with_vectorstore_retriever.invoke({'question':question_lists[1]})

"Being placed on academic probation means that students' behaviors, participation in class, timekeeping, or attendance did not meet the required standards expected at BUV. They will have to go through a process where their progress will be monitored and documented. If they successfully complete this process, they will be returned to normal academic status. If they are unable to successfully complete the process, their situation will be dealt with on a case-by-case basis by the Associate Professor and Dean (Higher Education). Thanks for asking!"

In [42]:
rag_chain_with_vectorstore_retriever.invoke({'question':question_lists[2]})

'If BUV students want to exchange or transfer to other universities around the world, they can contact the BUV International Office for more information and guidance. The International Office will provide advising on the transfer process, credits transfer, and necessary documents. Students are responsible for visa application and other procedures for their stay abroad. Thanks for asking!'

In [43]:
rag_chain_with_vectorstore_retriever.invoke({'question':question_lists[3]})

'Sinh viên BUV có thể tham gia chương trình Trao đổi và Chuyển tiếp sinh viên tới các trường Đại học trên thế giới. Trong chương trình Trao đổi, sinh viên có thể trải nghiệm môi trường học tập tại nước ngoài và tích lũy kinh nghiệm mới. Trong chương trình Chuyển tiếp, sinh viên có thể chuyển tiếp sang trường đại học mong muốn để hoàn thành khóa học và nhận bằng. Tuy nhiên, sinh viên cần liên hệ với Phòng Hợp tác Quốc tế của BUV để biết thêm thông tin về quy trình chuyển tiếp và chương trình trao đổi. Cả hai chương trình đều yêu cầu sinh viên tự chi trả các chi phí khác như vé máy bay, visa và chi phí sinh hoạt. Thanks for asking!'

In [44]:
rag_chain_with_vectorstore_retriever.stream({'question':question_lists[3]})

<generator object RunnableSequence.stream at 0x12e1a63e0>